In [30]:
import API_helper.url_module as hp
import pandas as pd
import time
import datetime
import os
from prefect.deployments import Deployment
from prefect import flow,task
from google.cloud import storage
from config import credential ,query_bq 
from prefect_gcp import GcpCredentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential.google_cred


@task(name='Get_BQ_SQL',log_prints=True)
def GetBQdata(query): 
    gcp_credentials_block = GcpCredentials.load("ete-projectcredential")
    df_bq = pd.read_gbq(query=query,
    project_id='ete-projectdatatalkclub',
    credentials=gcp_credentials_block.get_credentials_from_service_account()
    )
    return df_bq


@task(log_prints=True)
def getApiMap(para_regiontype,para_seriesgroup,para_season,para_unit,para_frequency,para_mindate,para_maxdate):

        print('Calling the API.')
        result = hp.map.regional(
            file_type ='json' \
            ,series_group = para_seriesgroup \
            ,region_type = para_regiontype \
            ,date = para_maxdate \
            ,start_date = para_mindate
            ,season=para_season \
            ,units=para_unit
            ,frequency = para_frequency
        )
        
        if result.ok:
            try:
                data_all = result.json()
                data_date =data_all['meta']['data']
                return data_date    

            except Exception:
                print(f'An error occured {Exception}')
                return None

        else:
            print(f'An error occur {result.status_code}.')
            print(f'An error occur {result.content}.')
            print('Waiting for 5 sec.')
            return 'retry'

@task(log_prints=True)
def movearchive(bucket,type):
    file_acrhive=  list(bucket.list_blobs())
    for file in file_acrhive:
        name = file.name
        pos = name.find('/')
        type = name[:pos]
        if type == 'staging':
            sub_pos = name.find('/',pos+1)
            sub_type = name[pos+1:sub_pos]
            if sub_type == type:
                bucket.copy_blob(file,destination_bucket=bucket,new_name=name.replace('staging','archive'))
                bucket.delete_blob(name)    

 



ImportError: cannot import name 'credential' from 'config' (d:\Avalon_Pratice_Code\ML_Project\ETE Project\Final_Project_FredETE\flows\config.py)

In [ ]:
@flow(log_prints=True)
def test():
    # series_parameter = getsqldata(query_sql.query_getMapPara)
    series_parameter = GetBQdata(query_bq.query_getMapPara)
    client = storage.Client()
    bucket = client.get_bucket('fred-etedata')
    print('Moving file to archive folder')
    movearchive(bucket,'map')
    time_stamp = datetime.datetime.now().strftime('%Y-%m-%d')
    for api_para in series_parameter.values.tolist():
        para_regiontype  = api_para[0][0]
        para_seriesgroup = api_para[1][0]
        para_season      = api_para[2][0]
        para_unit        = api_para[3][0]
        para_frequency   = api_para[4][0]   
        para_mindate     = api_para[5]
        para_maxdate     = api_para[6]

        retry = 0
        while retry <3:
            data_date =   getApiMap(para_regiontype,para_seriesgroup,para_season,para_unit, para_frequency,para_mindate,para_maxdate)
            if data_date == 'retry':
                retry+= 1
                print(f'Retry the current call. Current retry {retry}')
                continue
            elif data_date ==None:
                print(f'Skpping the call for {para_seriesgroup}')
                continue
            else:
                break
        
        if  retry ==3:
            print(f'Error occur skipping {para_seriesgroup}')
            continue
        time_para = para_maxdate.strftime('%Y-%m-%d')
        df_map = pd.json_normalize(data_date[time_para])
        
        
        uppload_path =f'staging/map/{time_stamp}/MapData_{para_seriesgroup}_{time_para}.parquet'
        print(f'Uploading file to cloud for {para_seriesgroup}')        
        bucket.blob(uppload_path).upload_from_string(df_map.to_parquet(), 'text/parquet')


        # retry = 0
        # while retry <2:
        #     try:
        #         bucket.blob(uppload_path).upload_from_string(df_map.to_parquet(), 'text/parquet')
        #         continue

        #     except Exception as err:
        #         retry+=1
        #         print(f'An error occur while upload file {err}')
        #         print(f'Wait for 5 second and retry. Current retry {retry}')
        #         time.sleep(5)
        time.sleep(1)
    print('Finish running the function.')
test()

In [ ]:
from dotenv import load_dotenv


basedir=os.getcwd()[:-5]
load_dotenv(os.path.join(basedir, './.env'))

In [36]:
from prefect_gcp.cloud_storage import GcsBucket

@flow(name='my_unique_name')
def test():
    gcp_cloud_storage_bucket_block = GcsBucket.load("ete-project-gcs")
    gcp_cloud_storage_bucket_block.UPLOAD_FROM_DATAFRAME()

In [37]:
test()

09:28:32.502 | INFO    | prefect.engine - Created flow run 'persimmon-partridge' for flow 'my_unique_name'

09:28:35.110 | ERROR   | Flow run 'persimmon-partridge' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\B\AppData\Local\Programs\Python\Python39\lib\site-packages\prefect\engine.py", line 643, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "c:\Users\B\AppData\Local\Programs\Python\Python39\lib\site-packages\prefect\utilities\asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "c:\Users\B\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "c:\Users\B\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\B\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Users\B\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "c:\Users\B\AppData\Local\Programs\Python\Python39\lib\site-packages\prefect\utilities\asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "C:\Users\B\AppData\Local\Temp/ipykernel_11608/1322335479.py", line 6, in test
    gcp_cloud_storage_bucket_block.UPLOAD_FROM_DATAFRAME()
AttributeError: 'GcsBucket' object has no attribute 'UPLOAD_FROM_DATAFRAME'

09:28:35.650 | ERROR   | Flow run 'persimmon-partridge' - Finished in state Failed("Flow run encountered an exception. AttributeError: 'GcsBucket' object has no attribute 'UPLOAD_FROM_DATAFRAME'\n")

AttributeError: 'GcsBucket' object has no attribute 'UPLOAD_FROM_DATAFRAME'

In [34]:
gcp_cloud_storage_bucket_block.UPLOAD_FROM_DATAFRAME()

AttributeError: 'coroutine' object has no attribute 'UPLOAD_FROM_DATAFRAME'

In [ ]:
os.getenv("Prefect_Credential")

In [31]:
from prefect_gcp.credentials import GcpCredentials
from prefect import flow,task
@flow()
def test():
    gcp_credentials_block = GcpCredentials.load("ete-projectcredential")
    gcp_credentials_block.get_credentials_from_service_account()
    print(gcp_credentials_block)

In [32]:
test()

09:22:24.766 | INFO    | prefect.engine - Created flow run 'quantum-beluga' for flow 'test'

GcpCredentials(service_account_file=None, service_account_info=SecretDict('{'type': '**********', 'auth_uri': '**********', 'client_id': '**********', 'token_uri': '**********', 'project_id': '**********', 'private_key': '**********', 'client_email': '**********', 'private_key_id': '**********', 'client_x509_cert_url': '**********', 'auth_provider_x509_cert_url': '**********'}'), project='ete-projectdatatalkclub')


09:22:27.627 | INFO    | Flow run 'quantum-beluga' - Finished in state Completed()

In [ ]:
def create_bigquery_client():
    gcp_credentials_block = GcpCredentials.load("BLOCK_NAME")
    google_auth_credentials = gcp_credentials_block.get_credentials_from_service_account()
    bigquery_client = bigquery.Client(credentials=google_auth_credentials)

In [25]:
GcpCredentials.get_credentials_from_service_account()

TypeError: get_credentials_from_service_account() missing 1 required positional argument: 'self'

In [ ]:
gcp_credentials_block.client_type()

In [ ]:
gcp_credentials_block.service_account_file()

In [20]:
from prefect_gcp.secret_manager import read_secret

In [21]:
from prefect_gcp import GcpCredentials
gcp_credentials_block = GcpCredentials.load("ete-projectcredential")

In [22]:
gcp_credentials_block

<coroutine object Block.load at 0x0000025992E448C0>

In [14]:
from prefect import flow
from prefect_gcp import GcpCredentials
from prefect_gcp.secret_manager import read_secret

@flow()
def example_cloud_storage_read_secret_flow():
    gcp_credentials = GcpCredentials.load("ete-projectcredential")
    secret_value = read_secret("secret_name", gcp_credentials, version_id=1)
    return secret_value

example_cloud_storage_read_secret_flow()


ValidationError: 2 validation errors for GcpSecret
gcp_credentials
  field required (type=value_error.missing)
secret_name
  field required (type=value_error.missing)